# Packages


In [ ]:
import numpy as np 
import matplotlib .pyplot as plt 
import time 
import zipfile 
import matplotlib 
from numba import njit 
from google .colab import files 
import matplotlib .colors as colors 
import matplotlib .patches as mpatches 
import matplotlib .collections as mcollections 
from IPython .display import clear_output 
import matplotlib .patches as patches 
from numba .core import types 
from numba .typed import List 
from scipy .stats import linregress ,pearsonr 
import pandas as pd 
import subprocess 
from skimage .measure import shannon_entropy 
import os 


# Results Processing


In [ ]:
# Takes as input an Nc x Nc (on-lattice) matrix and returns an N x 2 (off-lattice) matrix.
def On_to_Off (matrice_onlattice ):
  num_particelle =int (np .max (matrice_onlattice ))
  matrice_offlattice =np .zeros ((num_particelle ,2 ),dtype =np .float64 )
  occupied_coords =np .argwhere (matrice_onlattice >0 )
  for coords in occupied_coords :
    y ,x =coords [0 ],coords [1 ]
    particle_index =int (matrice_onlattice [y ,x ])
    matrice_offlattice [particle_index -1 ]=[x ,y ]
  return matrice_offlattice 

  # Takes as input an N x 2 (off-lattice) matrix and returns an Nc x Nc (on-lattice) matrix.
def Off_to_On (matrice_offlattice ):
  max_coord =np .max (matrice_offlattice )
  dimensione_matrice_iniziale =int (max_coord )+1 
  matrice_quadrata_iniziale =np .zeros ((dimensione_matrice_iniziale ,dimensione_matrice_iniziale ),dtype =int )
  min_x ,min_y =dimensione_matrice_iniziale ,dimensione_matrice_iniziale 
  max_x ,max_y =0 ,0 
  for i in range (matrice_offlattice .shape [0 ]):
    x ,y =matrice_offlattice [i ,0 ],matrice_offlattice [i ,1 ]
    matrice_quadrata_iniziale [int (y ),int (x )]=i +1 
    min_x =min (min_x ,int (x ))
    min_y =min (min_y ,int (y ))
    max_x =max (max_x ,int (x ))
    max_y =max (max_y ,int (y ))
  matrice_ritagliata =matrice_quadrata_iniziale [min_y :max_y +1 ,min_x :max_x +1 ]
  return matrice_ritagliata 

  # Save the matrices in off-lattice format.
def salvataggio (matrici ,tipo ,acceleratore ,transizione ,start ,output_dir ,kill =None ):
  nuove_matrici =[]
  for matrice in matrici :
    if tipo =='On-Lattice':
      N =np .max (matrici [0 ])
      nuove_matrici .append (On_to_Off (matrice ))
    if tipo =='Off-Lattice':
      N =matrici [0 ].shape [0 ]
      nuove_matrici .append (matrice )
  nuove_matrici =np .array (nuove_matrici )
  start =int (start )
  if kill is not None :kill =int (kill )
  if acceleratore =='Nessuno':filename_matrici =f"{tipo}_{acceleratore}_{int(N/1000)}k_start{int(start)}.npy"
  else :filename_matrici =f"{tipo}_{acceleratore}_{int(N/1000)}k_start{int(start)}_kill{int(kill)}.npy"
  percorso_completo =os .path .join (output_dir ,filename_matrici )
  np .save (percorso_completo ,nuove_matrici )
  print (f"Matrices file saved to: {percorso_completo}")

  # Save the table
def salvataggio_tabella (styler ,tipo ,acceleratore ,transizione ,start ,kill ,output_dir ,N ):
    filename =f"Tabella_{tipo}_{acceleratore}_{int(N/1000)}k_start{int(start)}_kill{int(kill)}_transition{transition}.html"
    percorso_completo =os .path .join (output_dir ,filename )
    styler .to_html (percorso_completo )
    print (f"Table saved to: {percorso_completo}")


# Cluster Visualization


In [ ]:
# Visualize a single simulation
def visualizzazione_singola (matrice ):
  if matrice .shape [0 ]==matrice .shape [1 ]or matrice .shape [1 ]!=2 :
    matrice =On_to_Off (matrice )

  all_x ,all_y =matrice [:,0 ],matrice [:,1 ]
  N =matrice .shape [0 ]
  all_indices =np .arange (N )
  cmap =matplotlib .colormaps ['viridis'].resampled (256 )
  norm =colors .Normalize (vmin =0 ,vmax =N )
  fig ,ax =plt .subplots (figsize =(8 ,8 ))
  ax .set_facecolor ('white')
  ax .set_aspect ('equal',adjustable ='box')
  cerchi =[mpatches .Circle ((x ,y ),radius =0.5 )for x ,y in zip (all_x ,all_y )]
  collection =mcollections .PatchCollection (cerchi ,cmap =cmap ,norm =norm ,linewidth =0.1 ,alpha =0.5 )
  collection .set_array (all_indices )
  ax .add_collection (collection )
  margine =10. 
  ax .set_xlim (all_x .min ()-margine ,all_x .max ()+margine )
  ax .set_ylim (all_y .min ()-margine ,all_y .max ()+margine )
  ax .invert_yaxis ()
  ax .set_aspect ('equal',adjustable ='box')
  plt .tight_layout (pad =0 )
  plt .show ()

  # Visualize all simulations from the same batch
def visualizzazione_multipla (matrici ):
  if matrici [0 ].shape [0 ]==matrici [0 ].shape [1 ]or matrici [0 ].shape [1 ]!=2 :
    nuove_matrici =[]
    for matrice_onlattice in matrici :
      matrice_offlattice =On_to_Off (matrice_onlattice )
      nuove_matrici .append (matrice_offlattice )
    matrici =nuove_matrici 

  Nsim =len (matrici )
  all_x ,all_y ,all_indices =[],[],[]
  N =matrici [0 ].shape [0 ]
  for matrix in matrici :
    indices =np .arange (N )
    all_x .append (matrix [:,0 ])
    all_y .append (matrix [:,1 ])
    all_indices .append (indices )
  all_x =np .concatenate (all_x )
  all_y =np .concatenate (all_y )
  all_indices =np .concatenate (all_indices )
  cmap =matplotlib .colormaps ['viridis'].resampled (256 )
  norm =colors .Normalize (vmin =0 ,vmax =N )
  fig ,ax =plt .subplots (figsize =(8 ,8 ))
  ax .set_facecolor ('white')
  ax .set_aspect ('equal',adjustable ='box')
  cerchi =[mpatches .Circle ((x ,y ),radius =0.5 )for x ,y in zip (all_x ,all_y )]
  collection =mcollections .PatchCollection (cerchi ,cmap =cmap ,norm =norm ,linewidth =0.1 ,alpha =0.5 )
  collection .set_array (all_indices )
  ax .add_collection (collection )
  margine =10. 
  ax .set_xlim (all_x .min ()-margine ,all_x .max ()+margine )
  ax .set_ylim (all_y .min ()-margine ,all_y .max ()+margine )
  ax .invert_yaxis ()
  ax .set_aspect ('equal',adjustable ='box')
  plt .tight_layout (pad =0 )
  plt .show ()


# Data Analysis


In [ ]:
######################### FRACTAL DIMENSION ##################################

def calcola_dimensione_Rg (punti ,plot =True ):# takes as input an off-lattice matrix.
    punti_fit =10 # number of points used for the fit.
    N_vec =np .logspace (np .log10 (1000 ),np .log10 (len (punti )),punti_fit )# avoid using too small particle counts.
    Rg_vec =np .zeros (len (N_vec ))
    for i ,n in enumerate (N_vec ):
      centroide =np .mean (punti [:int (n )],axis =0 )
      Rg_vec [i ]=np .sqrt (np .sum ((punti [:int (n )]-centroide )**2 )/int (n ))
    y =np .log (N_vec )
    x =np .log (Rg_vec )
    slope ,intercept ,*_ =linregress (x ,y )

    if plot :
        plt .figure (figsize =(6 ,4 ))
        plt .plot (x ,y ,'o',label ='Simulation data')
        plt .plot (x ,slope *x +intercept ,'r-',label =f'Fit lineare (D = {slope:.3f})')
        plt .xlabel ('log(Rg)')
        plt .ylabel ('log(N)')
        plt .title ('Radius of Gyration Method')
        plt .legend ()
        plt .grid (True )
        plt .tight_layout ()
        plt .show ()

    return slope 

def calcola_dimensione_massradius (punti ,plot =True ):# takes as input an off-lattice matrix.
    N =punti .shape [0 ]
    centroide =np .mean (punti ,axis =0 )
    distanze =np .hypot (punti [:,0 ]-centroide [0 ],punti [:,1 ]-centroide [1 ])
    Rmax =np .max (distanze )
    punti_fit =10 # number of points used for the fit.
    # Choose a radius interval that excludes values that are too small (few particles) or too large (cluster still forming).
    raggi_vec =np .logspace (np .log10 (Rmax *0.05 ),np .log10 (Rmax *0.7 ),punti_fit )
    conteggi =np .array ([np .sum (distanze <=r )for r in raggi_vec ])
    x =np .log (raggi_vec )
    y =np .log (conteggi )
    slope ,intercept ,*_ =linregress (x ,y )

    if plot :
        plt .plot (x ,y ,'o',label ='Simulation data')
        plt .plot (x ,slope *x +intercept ,'r-',label =f'Fit lineare (D = {slope:.3f})')
        plt .xlabel ('log(R)')
        plt .ylabel ('log(N)')
        plt .title ('Mass–Radius Method')
        plt .legend ()
        plt .grid (True )
        plt .tight_layout ()
        plt .show ()

    return slope 

def calcola_dimensione_boxcounting (punti ,plot =True ):# takes as input an off-lattice matrix.
  punti_fit =10 
  min_vals =np .min (punti ,axis =0 )
  max_vals =np .max (punti ,axis =0 )
  cluster_size =np .max (max_vals -min_vals )
  min_eps =20. # exclude boxes whose size is too small compared to the distance between adjacent particles.
  max_eps =cluster_size /16. # exclude boxes that are too large, which would all be fully occupied.
  epsilons =np .logspace (np .log10 (min_eps ),np .log10 (max_eps ),punti_fit )
  N_particelle =np .zeros (len (epsilons ))
  for i ,eps in enumerate (epsilons ):
    grid_indices =np .floor ((punti -min_vals )/eps ).astype (int )
    unique_boxes =np .unique (grid_indices ,axis =0 )
    N_particelle [i ]=len (unique_boxes )
    x =np .log (1 /epsilons )
    y =np .log (N_particelle )
    slope ,intercept ,*_ =linregress (x ,y )

  if plot :
    plt .plot (x ,y ,'o',label ="Data")
    plt .plot (x ,slope *x +intercept ,'r-',label =f"Fit lineare (D = {slope:.3f})")
    plt .xlabel ("log(1/ε)")
    plt .ylabel ("log N(ε)")
    plt .title ("Box-counting method")
    plt .legend ()
    plt .grid (True )
    plt .tight_layout ()
    plt .show ()

  return slope 

  ########################### COMPLEXITY ########################################

def calcola_complessita (matrice ):
# If the matrix is in off-lattice format, convert it to on-lattice format.
  if matrice .shape [0 ]!=matrice .shape [1 ]or matrice .shape [1 ]==2 :
    matrice =Off_to_On (matrice )
  matrice_binaria =(matrice >0 ).astype (np .uint8 )*255 
  S =shannon_entropy (matrice_binaria )
  return S *(1 -S )

  ############################ ANISOTROPY #######################################

def calcola_anisotropia (matrice ):# takes as input an off-lattice matrix.
  N =matrice .shape [0 ]
  centro_di_massa =np .mean (matrice ,axis =0 )
  particelle_centrate =matrice -centro_di_massa 
  matrice_covarianza =np .cov (particelle_centrate ,rowvar =False )
  autovalori ,autovettori =np .linalg .eigh (matrice_covarianza )
  anisotropia =autovalori [1 ]/autovalori [0 ]
  angolo =np .arctan2 (autovettori [1 ,1 ],autovettori [0 ,1 ])
  return anisotropia ,np .cos (4 *angolo )


# Simulation


In [ ]:
@njit 
def codifica (tipo ,acceleratore ):
  if tipo =='On-Lattice':a =0 
  elif tipo =='Off-Lattice':a =1 

  if acceleratore =='Nessuno':b =0 
  elif acceleratore =='Standard':b =1 
  elif acceleratore =='Propagatore':b =2 
  elif acceleratore =='Meakin':b =3 

  return a *10 +b 

@njit 
def decodifica (codice ):
    a =codice //10 
    b =codice %10 
    return a ,b 

    ################################################################################

@njit 
def inizializza_griglia_vicinanza (N ,L ):
  grid_dim =int (L )+1 
  grid =np .full ((grid_dim ,grid_dim ),-1 ,dtype =np .int32 )
  next_in_cell =np .full (N +1 ,-1 ,dtype =np .int32 )
  return grid ,next_in_cell 

@njit 
def aggiorna_griglia_vicinanza (indice_particella ,matrice ,grid ,next_in_cell ):
    px ,py =matrice [indice_particella ,0 ],matrice [indice_particella ,1 ]
    col =int (px )
    row =int (py )
    next_in_cell [indice_particella ]=grid [row ,col ]
    grid [row ,col ]=indice_particella 

    ################################################################################

def crea_mappe_gerarchiche (lato ):
  mappe =[]
  numero_celle =lato 
  while numero_celle >1 :
    numero_celle =(numero_celle +1 )//2 
    mappe .append (np .full ((numero_celle ,numero_celle ),False ,dtype =bool ))
  mappe_numba =List .empty_list (types .boolean [:,:])
  for arr in mappe :
    mappe_numba .append (arr )
  return mappe_numba 

@njit 
def aggiorna_mappe_gerarchiche (mappe ,x ,y ):
  col ,row =int (x ),int (y )
  for livello in range (len (mappe )):
    row //=2 
    col //=2 
    if mappe [livello ][row ,col ]:return 
    else :mappe [livello ][row ,col ]=True 

@njit 
def trova_distanza_salto (mappe ,x ,y ,stepsize ):
  distanza_salto =stepsize 
  livello_b ,livello_a =int (x ),int (y )
  for livello in range (len (mappe )):
    dimensione_box =2 **(livello +1 )
    livello_a //=2 
    livello_b //=2 
    occupato =False 
    for i in range (-1 ,2 ):
      for j in range (-1 ,2 ):
        shape_a ,shape_b =mappe [livello ].shape 
        check_a =max (0 ,min (livello_a +i ,shape_a -1 ))
        check_b =max (0 ,min (livello_b +j ,shape_b -1 ))
        if mappe [livello ][check_a ,check_b ]:
          occupato =True 
          break 
      if occupato :
        break 
    if occupato :
      return distanza_salto 
    distanza_salto =dimensione_box /2.0 

  return distanza_salto 

@njit 
def inizializza_mappe_gerarchiche_offlattice (matrice ,mappe ):
  for i in range (matrice .shape [0 ]):
    if matrice [i ,0 ]>0 and matrice [i ,1 ]>0 :# if a particle coordinate is 0, then it is touching the "borders" (not allowed), or it has not been inserted yet.
      aggiorna_mappe_gerarchiche (mappe ,matrice [i ,0 ],matrice [i ,1 ])
  return mappe 

def inizializza_mappe_gerarchiche_onlattice (matrice ,mappe ):
  particelle =np .argwhere (matrice >0 )
  for i in range (particelle .shape [0 ]):
    y ,x =particelle [i ]
    aggiorna_mappe_gerarchiche (mappe ,x ,y )
  return mappe 

  ################################################################################

@njit 
def inizializza_posizione (centro ,Rc ,start ):
  theta =2 *np .pi *np .random .random ()
  x =(Rc +start )*np .cos (theta )+centro 
  y =(Rc +start )*np .sin (theta )+centro 
  return x ,y 

@njit 
def contatto_offlattice (matrice ,x ,y ,grid ,next_in_cell ):
  ix =int (x )
  iy =int (y )
  grid_dim =grid .shape [0 ]
  # Check distances between the particle under consideration and those in the 9 neighboring cells.
  for i in range (max (0 ,ix -1 ),min (grid_dim ,ix +2 )):
    for j in range (max (0 ,iy -1 ),min (grid_dim ,iy +2 )):
      indice_particella =grid [j ,i ]
      while indice_particella !=-1 :
        px ,py =matrice [indice_particella ,0 ],matrice [indice_particella ,1 ]
        if (x -px )**2 +(y -py )**2 <1.0 :return True 
        indice_particella =next_in_cell [indice_particella ]
  return False 

@njit 
def evolvi_posizione_offlattice (matrice ,step ,x ,y ,Rc ,start ,kill ,stepsize ,grid ,next_in_cell ,mappe ,b ,c ):
  centro =matrice [0 ,0 ]
  L =centro *2 
  while not contatto_offlattice (matrice ,x ,y ,grid ,next_in_cell ):
  # Perform the random-walk step, optionally augmented with hierarchical maps.
    if c ==0 :distanza =stepsize 
    if c ==1 :distanza =trova_distanza_salto (mappe ,x ,y ,stepsize )
    theta =2 *np .pi *np .random .random ()
    x +=distanza *np .cos (theta )
    y +=distanza *np .sin (theta )

    # Apply the appropriate accelerator, if enabled.
    if b !=0 :
      x_rel ,y_rel =x -centro ,y -centro 
      r0 =np .hypot (x_rel ,y_rel )
      if r0 >Rc +kill :# Standard accelerator
        if b ==1 :x ,y =inizializza_posizione (centro ,Rc ,start )
        if b ==2 :# Propagator accelerator
          Rrestart =Rc +1 
          V =(r0 -Rrestart )/(r0 +Rrestart )*np .tan (np .pi *(np .random .rand ()-0.5 ))
          x ,y =Rrestart /r0 *((1 -V **2 )*x_rel -2 *V *y_rel )/(1 +V **2 )+centro ,Rrestart /r0 *((1 -V **2 )*y_rel +2 *V *x_rel )/(1 +V **2 )+centro 
        if b ==3 :# Meakin accelerator
          while r0 >Rc +kill :
            theta =2 *np .pi *np .random .random ()
            x +=(r0 -Rc -start )*np .cos (theta )
            y +=(r0 -Rc -start )*np .sin (theta )
            if x >=L :x =L -1e-9 
            if x <0 :x =0 
            if y >=L :y =L -1e-9 
            if y <0 :y =0 
            r0 =np .hypot (x -centro ,y -centro )

            # Safety clamping before the while-condition check.
    if x >=L :x =L -1e-9 
    if x <0 :x =0 
    if y >=L :y =L -1e-9 
    if y <0 :y =0 

  return x ,y 

@njit 
def evolvi_posizione_onlattice (matrice ,x ,y ,Rc ,start ,kill ,stepsize ,b ,c ,mappe ):
  centro =matrice .shape [0 ]//2 
  Nc =matrice .shape [0 ]
  while matrice [y ,x +1 ]==0 and matrice [y ,x -1 ]==0 and matrice [y +1 ,x ]==0 and matrice [y -1 ,x ]==0 :
  # Save the starting position for a final check.
    x_old ,y_old =x ,y 
    # Perform the random-walk step, optionally augmented with hierarchical maps.
    if c ==0 :
      r =np .random .randint (0 ,4 )
      if r ==0 and x <Nc -2 :x +=1 
      elif r ==1 and x >1 :x -=1 
      elif r ==2 and y <Nc -2 :y +=1 
      elif r ==3 and y >1 :y -=1 
    if c ==1 :
      distanza =trova_distanza_salto (mappe ,x ,y ,stepsize )
      theta =2 *np .pi *np .random .random ()
      x +=round (distanza *np .cos (theta ))
      y +=round (distanza *np .sin (theta ))

      # Apply the appropriate accelerator, if enabled.
    if b !=0 :
      x_rel ,y_rel =x -centro ,y -centro 
      r0 =np .hypot (x_rel ,y_rel )
      if r0 >Rc +kill :# Standard accelerator
        if b ==1 :
          x ,y =inizializza_posizione (centro ,Rc ,start )
          x ,y =int (x ),int (y )
        if b ==2 :# Propagator accelerator
          Rrestart =Rc +1 
          V =(r0 -Rrestart )/(r0 +Rrestart )*np .tan (np .pi *(np .random .rand ()-0.5 ))
          x =round (Rrestart /r0 *((1 -V **2 )*x_rel -2 *V *y_rel )/(1 +V **2 )+centro )
          y =round (Rrestart /r0 *((1 -V **2 )*y_rel +2 *V *x_rel )/(1 +V **2 )+centro )
        if b ==3 :# Meakin accelerator
          while r0 >Rc +kill :
            theta =2 *np .pi *np .random .random ()
            x +=round ((r0 -Rc -start )*np .cos (theta ))
            y +=round ((r0 -Rc -start )*np .sin (theta ))
            if x >=Nc -1 :x =Nc -2 
            if x <1 :x =1 
            if y >=Nc -1 :y =Nc -2 
            if y <1 :y =1 
            r0 =np .hypot (x -centro ,y -centro )

            # Safety clamping before the while-condition check.
            # The particle must not leave the simulation domain.
    if x >=Nc -1 :x =Nc -2 
    if x <1 :x =1 
    if y >=Nc -1 :y =Nc -2 
    if y <1 :y =1 
    # The particle must not end up in an already occupied cell.
    if matrice [y ,x ]!=0 :x ,y =x_old ,y_old 

  return x ,y 

  ################################################################################

def simulazione_offlattice (N ,b ,transizione ,start ,kill =0 ,stepsize =1. ):
# Choose appropriate domain dimensions.
  if b ==0 :# Accelerator "Nessuno" (none): choose a domain that is not too large, otherwise the simulation is extremely slow.
  # A DLA cluster with 3000 particles has a typical radius of 90. To leave room for diffusion we multiply by 2 and apply a scaling law.
    L =int (2 *(90 *2 )*(N /3000 )**(2 /3 ))*1. 
  if b !=0 :# Accelerator enabled: we can choose a larger domain, since everything will be cropped by kill.
    L =int (2 *(90 *5 )*(N /3000 )**(2 /3 ))*1. 
  centro =L /2 
  lato =int (L )

  # Define the fundamental data structures.
  matrice =np .zeros ((N ,2 ),dtype =np .float64 );matrice [0 ]=[centro ,centro ]
  grid ,next_in_cell =inizializza_griglia_vicinanza (N ,L )
  aggiorna_griglia_vicinanza (0 ,matrice ,grid ,next_in_cell )
  tempi =[]
  Rc =10. 

  # Start the simulation without hierarchical maps.
  c =0 
  mappe =List .empty_list (types .boolean [:,:])
  start_time =time .time ()
  for step in range (1 ,transizione ):
    x ,y =inizializza_posizione (centro ,Rc ,start )
    # Evolve the particle position and update the data structures.
    x_finale ,y_finale =evolvi_posizione_offlattice (matrice ,step ,x ,y ,Rc ,start ,kill ,stepsize ,grid ,next_in_cell ,mappe ,b ,c )
    matrice [step ]=[x_finale ,y_finale ]
    aggiorna_griglia_vicinanza (step ,matrice ,grid ,next_in_cell )
    Rc =max (Rc ,np .hypot (x_finale -centro ,y_finale -centro ))
    # Checkpoint.
    if (10 *(step +1 ))%N ==0 :
      end_time =time .time ()
      tempi .append (end_time -start_time )
      print (f'Particelle: {step+1} di {N}. Ultimo aggiornamento in {(end_time-start_time):.1f} secondi.')
      start_time =time .time ()

  start_time =time .time ()
  # Build the hierarchical maps.
  mappe =crea_mappe_gerarchiche (lato )
  inizializza_mappe_gerarchiche_offlattice (matrice ,mappe )

  # Resume the simulation with hierarchical maps.
  c =1 
  start_time =time .time ()
  for step in range (transizione ,N ):
    x ,y =inizializza_posizione (centro ,Rc ,start )
    # Evolve the particle position and update the data structures.
    x_finale ,y_finale =evolvi_posizione_offlattice (matrice ,step ,x ,y ,Rc ,start ,kill ,stepsize ,grid ,next_in_cell ,mappe ,b ,c )
    matrice [step ]=[x_finale ,y_finale ]
    aggiorna_griglia_vicinanza (step ,matrice ,grid ,next_in_cell )
    Rc =max (Rc ,np .hypot (x_finale -centro ,y_finale -centro ))
    aggiorna_mappe_gerarchiche (mappe ,x_finale ,y_finale )
    # Checkpoint.
    if (10 *(step +1 ))%N ==0 :
      end_time =time .time ()
      tempi .append (end_time -start_time )
      print (f'Particelle: {step+1} di {N}. Ultimo aggiornamento in {(end_time-start_time):.1f} secondi.')
      start_time =time .time ()

  return matrice ,tempi 

def simulazione_onlattice (N ,b ,transizione ,start ,kill =0 ,stepsize =1 ):
# Choose appropriate domain dimensions.
  if b ==0 :# Accelerator "Nessuno" (none): choose a domain that is not too large, otherwise the simulation is extremely slow.
  # A DLA cluster with 3000 particles has a typical radius of 100. To leave room for diffusion we multiply by 2 and apply a scaling law.
    Nc =int (2 *(100 *2 )*(N /3000 )**(2 /3 ))|1 
  if b !=0 :# Accelerator enabled: we can choose a larger domain, since everything will be cropped by kill.
    Nc =int (2 *(100 *5 )*(N /3000 )**(2 /3 ))|1 
  centro =Nc //2 
  lato =Nc 

  # Define the fundamental data structures.
  matrice =np .zeros ((Nc ,Nc ));matrice [centro ,centro ]=1 
  tempi =[]
  Rc =10. 

  # Start the simulation without hierarchical maps.
  c =0 
  mappe =List .empty_list (types .boolean [:,:])
  start_time =time .time ()
  for step in range (1 ,transizione ):
    x ,y =inizializza_posizione (centro ,Rc ,start )
    x ,y =int (x ),int (y )
    # Evolve the particle position and update the data structures.
    x_finale ,y_finale =evolvi_posizione_onlattice (matrice ,x ,y ,Rc ,start ,kill ,stepsize ,b ,c ,mappe )
    matrice [y_finale ,x_finale ]=step +1 
    Rc =max (Rc ,np .hypot (x_finale -centro ,y_finale -centro ))
    if (Rc +start >centro and b ==0 )or (Rc +kill >centro and b !=0 ):raise Exception ("Increase the size of the on-lattice grid.")
    # Checkpoint.
    if (10 *(step +1 ))%N ==0 :
      end_time =time .time ()
      tempi .append (end_time -start_time )
      print (f'Particelle: {step+1} di {N}. Ultimo aggiornamento in {(end_time-start_time):.1f} secondi.')
      start_time =time .time ()

  start_time =time .time ()
  # Build the hierarchical maps.
  mappe =crea_mappe_gerarchiche (lato )
  inizializza_mappe_gerarchiche_onlattice (matrice ,mappe )

  # Resume the simulation with hierarchical maps.
  c =1 
  start_time =time .time ()
  for step in range (transizione ,N ):
    x ,y =inizializza_posizione (centro ,Rc ,start )
    x ,y =int (x ),int (y )
    # Evolve the particle position and update the data structures.
    x_finale ,y_finale =evolvi_posizione_onlattice (matrice ,x ,y ,Rc ,start ,kill ,stepsize ,b ,c ,mappe )
    matrice [y_finale ,x_finale ]=step +1 
    Rc =max (Rc ,np .hypot (x_finale -centro ,y_finale -centro ))
    if (Rc +start >centro and b ==0 )or (Rc +kill >centro and b !=0 ):raise Exception ("Increase the size of the on-lattice grid.")
    aggiorna_mappe_gerarchiche (mappe ,x_finale ,y_finale )
    # Checkpoint.
    if (10 *(step +1 ))%N ==0 :
      end_time =time .time ()
      tempi .append (end_time -start_time )
      print (f'Particelle: {step+1} di {N}. Ultimo aggiornamento in {(end_time-start_time):.1f} secondi.')
      start_time =time .time ()

  return matrice ,tempi 

  ################################################################################


def simulazione (N ,codice ,transizione ,start ,kill =0 ,stepsize =1. ):
  a ,b =decodifica (codice )
  if a ==0 :matrice ,tempi =simulazione_onlattice (N ,b ,transizione ,start ,kill ,stepsize )
  if a ==1 :matrice ,tempi =simulazione_offlattice (N ,b ,transizione ,start ,kill ,stepsize )
  return matrice ,tempi 


# Data Collection


In [ ]:
def raccolta_dati (Nsim ,N ,tipo ,acceleratore ,mappe_gerarchiche ,transizione ,start ,output_dir ,kill =0 ,stepsize =1. ):
    codice =codifica (tipo ,acceleratore )
    if (tipo !='On-Lattice'and tipo !='Off-Lattice')or (acceleratore !='Nessuno'and acceleratore !='Standard'and acceleratore !='Propagatore'and acceleratore !='Meakin'):
        raise Exception ("Choose a valid accelerator or a valid type.")
    if (mappe_gerarchiche =="Si"and transizione >=N )or (mappe_gerarchiche =="No"and transizione <N ):
        raise Exception ("Review the choices for hierarchical maps.")

        # ... (your code for running the simulations and creating the DataFrame remains IDENTICAL) ...
    risultati_simulazioni =[]
    matrici_simulazioni =[]
    print (f"Running {Nsim} DLA {tipo} simulations of {int(N/1000)}k particles with accelerator {acceleratore} and transition {transizione}.")
    for i in range (Nsim ):
        print (f"\nSimulation {i+1}.")
        matrice_risultato ,tempi_risultato =simulazione (N ,codice ,transizione ,start ,kill ,stepsize )
        matrici_simulazioni .append (matrice_risultato )
        tempo_totale_simulazione =np .sum (tempi_risultato )

        if matrice_risultato .shape [0 ]==matrice_risultato .shape [1 ]or matrice_risultato .shape [1 ]!=2 :
          matrice_risultato =On_to_Off (matrice_risultato )

        D_gyration =calcola_dimensione_Rg (matrice_risultato ,plot =False )
        D_massradius =calcola_dimensione_massradius (matrice_risultato ,plot =False )
        D_boxcounting =calcola_dimensione_boxcounting (matrice_risultato ,plot =False )
        complexity =calcola_complessita (matrice_risultato )
        anisotropia ,coseno_angolo =calcola_anisotropia (matrice_risultato )
        sospetta =False 
        if D_gyration >1.80 or D_gyration <1.62 :
          print (f"Simulation {i+1} flagged")
          visualizzazione_singola (matrice_risultato )
          sospetta =True 

        risultati_simulazioni .append ({
        'Simulation':i +1 ,'Execution Time (s)':tempo_totale_simulazione ,
        'Radius of Gyration Method':D_gyration ,'Mass–Radius Method':D_massradius ,
        'Box-Counting Method':D_boxcounting ,'Complexity':complexity ,
        'Anisotropy':anisotropia ,'Q4 Parameter':coseno_angolo ,'Flagged':sospetta 
        })
        ################################################################################
    print ("\n")
    df_risultati =pd .DataFrame (risultati_simulazioni )
    cols_to_average =['Radius of Gyration Method','Mass–Radius Method']
    df_risultati ['Mean D2 per simulation']=df_risultati [cols_to_average ].mean (axis =1 )
    cols_for_stats =['Execution Time (s)','Box-Counting Method','Radius of Gyration Method',
    'Mass–Radius Method','Mean D2 per simulation','Complexity','Anisotropy','Q4 Parameter']
    mean_row_data ={col :df_risultati [col ].mean ()for col in cols_for_stats }
    std_dev_row_data ={col :df_risultati [col ].std ()/np .sqrt (Nsim )for col in cols_for_stats }
    mean_row =pd .DataFrame ([mean_row_data ]);mean_row ['Simulation']='Mean across simulations'
    std_dev_row =pd .DataFrame ([std_dev_row_data ]);std_dev_row ['Simulation']='Standard error of the mean'
    df_visualizzazione =df_risultati .drop (columns =['Flagged'])
    df_finale =pd .concat ([df_visualizzazione ,mean_row ,std_dev_row ],ignore_index =True )
    colonne_ordinate =['Simulation','Execution Time (s)','Box-Counting Method','Radius of Gyration Method',
    'Mass–Radius Method','Mean D2 per simulation','Complexity','Anisotropy','Q4 Parameter']
    df_finale =df_finale [colonne_ordinate ]
    def get_hardware_info ():
        try :
            result =subprocess .run (['nvidia-smi','--query-gpu=name','--format=csv,noheader'],capture_output =True ,text =True ,check =True )
            return result .stdout .strip ()
        except (FileNotFoundError ,subprocess .CalledProcessError ):
            try :
                result =subprocess .run (['lscpu'],capture_output =True ,text =True ,check =True )
                for line in result .stdout .split ('\n'):
                    if 'Model name:'in line :return line .split (':')[1 ].strip ()
                return "CPU (info unavailable)"
            except (FileNotFoundError ,subprocess .CalledProcessError ):return "Hardware not detected"
    hardware_usato =get_hardware_info ()
    titolo_tabella =(f"N = {N}, Type = '{tipo}', Accelerator = '{acceleratore}', start = {int(start)}, kill = {int(kill)}, stepsize = {int(stepsize)}, transition = {transizione}\nExecuted on: {hardware_usato}")
    format_mapping ={'Execution Time (s)':'{:.1f}','Box-Counting Method':'{:.3f}','Radius of Gyration Method':'{:.3f}',
    'Mass–Radius Method':'{:.3f}','Mean D2 per simulation':'{:.3f}','Complexity':'{:.5f}',
    'Anisotropy':'{:.2f}','Q4 Parameter':'{:.2f}'}
    col_idx ={nome :i +1 for i ,nome in enumerate (df_finale .columns ,1 )}
    idx_ultima_sim_riga =len (df_risultati )
    styler =df_finale .style .format (format_mapping ,na_rep ="-").set_caption (titolo_tabella )
    styler .set_table_styles ([{'selector':'caption','props':[('caption-side','top'),('font-size','1.1em'),('font-weight','bold'),('text-align','center'),('margin-bottom','10px')]},
    {'selector':'th, td','props':[('text-align','center')]},{'selector':'thead tr','props':[('border-bottom','2px solid black')]},
    {'selector':f'tbody tr:nth-child({idx_ultima_sim_riga})','props':[('border-bottom','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Simulation']}), td:nth-child({col_idx['Simulation']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Execution Time (s)']}), td:nth-child({col_idx['Execution Time (s)']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Box-Counting Method']}), td:nth-child({col_idx['Box-Counting Method']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Mass–Radius Method']}), td:nth-child({col_idx['Mass–Radius Method']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Mean D2 per simulation']}), td:nth-child({col_idx['Mean D2 per simulation']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Complexity']}), td:nth-child({col_idx['Complexity']})",'props':[('border-right','2px solid black')]},
    {'selector':f"th:nth-child({col_idx['Anisotropy']}), td:nth-child({col_idx['Anisotropy']})",'props':[('border-right','2px solid black')]},],overwrite =False )
    def highlight_sospetta (row ):
        if row .name <len (df_risultati )and df_risultati .loc [row .name ,'Flagged']:return ['color: red']*len (row )
        return ['']*len (row )
    styler .apply (highlight_sospetta ,axis =1 )
    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    clear_output ()
    display (styler )

    df_filtrato =df_risultati [df_risultati ['Flagged']==False ]
    print ("\n"+"="*60 )
    print ("Analysis computed EXCLUDING flagged simulations")
    print ("="*60 )

    valori_filtrati =df_filtrato ['Mean D2 per simulation']
    if not valori_filtrati .empty :
      media_filtrata =valori_filtrati .mean ()
      errore_filtrato =valori_filtrati .std ()/np .sqrt (len (valori_filtrati ))
      print (f"Numero di simulazioni valide considerate: {len(df_filtrato)}")
      print (f"D2 medio (filtrato): {media_filtrata:.3f} ± {errore_filtrato:.3f}")
    else :
      print ("No valid simulation to analyze.")

    salvataggio (matrici_simulazioni ,tipo ,acceleratore ,transizione ,start ,output_dir ,kill )
    salvataggio_tabella (styler ,tipo ,acceleratore ,transizione ,start ,kill ,output_dir ,N )


# Main
